In [1]:
# basic imports
import pandas as pd
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import scanpy as sc

import torch
import scipy
import matplotlib.patches as mpatches
from IPython.display import display
import seaborn as sns

# torch imports
from torch.nn.functional import softmax, cosine_similarity, sigmoid

In [3]:
# # add `Tangram` to path
# import sys
# sys.path.append("./Tangram") 

# # Tangram imports
# import mapping.utils
# import mapping.mapping_optimizer
# import mapping.plot_utils

In [4]:
'Folder where data are stored'
fld = 'data/'
assert os.path.isdir(fld)

In [5]:
'Load snRNA-seq data (AnnData format)'
path = os.path.join(fld, 'visp_sn_tpm_small_0430.h5ad')
adata = sc.read_h5ad(path)
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 11759 × 40056
    obs: 'consensus_cluster', 'reference_cluster', 'reference_color', 'broad_class', 'subclass', 'merged_cluster_smFISH', 'merged_cluster_ISS', 'total_reads', 'cpm_scaling_factor', 'n_genes'
    var: 'length', 'maxMeanFpkm', 'in_MERFISH', 'gene_id', 'chromosome', 'gene_entrez_id', 'gene_name', 'n_cells'

In [6]:
adata.obs

,consensus_cluster,reference_cluster,reference_color,broad_class,subclass,merged_cluster_smFISH,merged_cluster_ISS,total_reads,cpm_scaling_factor,n_genes
index,,,,,,,,,,
LS-14690_S05_E1-50,cl108_e49_NA_L4,VISp L4 Rspo1,#00979D,Ex,L4,L4,VISp L4 Rspo1,1749059.0,1.749059,9530
LS-14690_S07_E1-50,cl1_e421_Cst3_L4,VISp L4 Rspo1,#00979D,Ex,L4,L4,VISp L4 Rspo1,1674771.0,1.674771,9119
LS-14690_S08_E1-50,cl163_e33_Nxf7_L4,VISp L4 Rspo1,#00979D,Ex,L4,L4,VISp L4 Rspo1,1726516.0,1.726516,10834
LS-14690_S09_E1-50,cl1_e421_Cst3_L4,VISp L4 Rspo1,#00979D,Ex,L4,L4,VISp L4 Rspo1,1449575.0,1.449575,8889
LS-14690_S10_E1-50,cl5_i231_Nov_Vip,Vip Htr1f,#FF4DC1,Inh,Vip,Vip_3,Vip Htr1f,1450726.0,1.450726,8389
...,...,...,...,...,...,...,...,...,...,...
US-1250275_E1_S49,cl46_e95_Npc1l1_L5_IT,VISp L5 IT Pld5,#26BF64,Ex,L5 IT,L5 IT,VISp L5 IT Pld5,803378.0,0.803378,10800
US-1250275_E1_S57,cl46_e95_Npc1l1_L5_IT,VISp L5 IT Pld5,#26BF64,Ex,L5 IT,L5 IT,VISp L5 IT Pld5,905773.0,0.905773,10814
US-1250275_E1_S71,cl46_e95_Npc1l1_L5_IT,VISp L5 IT Pld5,#26BF64,Ex,L5 IT,L5 IT,VISp L5 IT Pld5,960667.0,0.960667,11366


In [5]:
path = os.path.join(fld, 'smFISH_anndata.h5ad')
smFiSH_anndata = sc.read_h5ad(path)
smFiSH_anndata

OSError: Unable to open file (file signature not found)

In [7]:
'Load spatial Visium data (AnnData format)'
path = os.path.join(fld, 'Allen-Visium_Allen1_cell_count.h5ad')
space_data = sc.read_h5ad(path)
space_data

AnnData object with n_obs × n_vars = 2669 × 31053
    obs: 'barcodes', 'under_tissue', '_x', '_y', 'x', 'y', 'xy', 'n_counts', 'sample', 'x_plot', 'y_plot', 'cell_count'
    var: 'n_counts'
    uns: 'fiducial_diameter_fullres', 'image_hires', 'spot_diameter_fullres', 'tissue_hires_scalef'
    obsm: 'proportions_class', 'proportions_cluster'